## Importing Required Libraries and dependencies.




In [169]:
#Import nessesary modules
import yfinance as yf
import pandas as pd
import json 
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import scipy.optimize as sci_plt
import streamlit as st

from pprint import pprint
from sklearn.preprocessing import StandardScaler

# Set display options for Pandas
# pd.set_option('display.max_colwidth', None)
# pd.set_option('expand_frame_repr', False)


## Step 1: Propt the user to customize choice for analysis

### 1.1 Prompt the user about how much capital they wish to invest??? Is it needed or we can calculate mean() for each dollar invested ?
### !!! Create widget on Streamlit for the prompt!!!

In [170]:
#Ask user how much is the allocation for Equity Allocation
# Check the input 
while True:
    try:
        equity_allocation = float(input('Please enter the amount of capital you wish to invest :'))
        break
    except:
        print('Please enter a numerical value.')


Please enter the amount of capital you wish to invest : 10000


## 1.2 Prompt the user how many years of historical data they wish to pull for the analysis.
### !!! Create widget on Streamlit for the prompt!!!

In [171]:
# Prompt the user how many years of historical data they wish to pull for the analysis
# Check the input 
while True:
    try:
        num_years = int(input('Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis :'))
        break
    except:
        print('Please enter a numerical value.')


Please enter the number of years (1-10) of historical data that you wish to retrieve for the analysis : 4


## 1.3 Prompt the user to select assets from each asset class.
### ??? Does the number of assets in each asset class should be the same for correct analysis???

### !!! Create widget on Streamlit for the prompt!!!

## Step 2 : 
### Data Collection and Cleaning
The first step is to collect data for the 3 different assets - crypto, commodities, and stocks. Using **yfinance** library for Pandas pull data from the Yahoo finance API. The data pulled for each asset class would have 10 tickers. Store the data for each asset class in separate Pandas DataFrames.

In [172]:
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded. 
### VARIATION 1
# crypto = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
# stocks = ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
# commodities = ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']

# # Collect data for crypto asset class
# crypto_df = pd.DataFrame()
# for ticker in crypto:
#     crypto_df[ticker] = yf.download(ticker)['Close']

# # Collect data for stocks asset class
# stocks_df = pd.DataFrame()
# for ticker in stocks:
#     stocks_df[ticker] = yf.download(ticker)['Close']

# # Collect data for commodities asset class
# commodities_df = pd.DataFrame()
# for ticker in commodities:
#     commodities_df[ticker] = yf.download(ticker)['Close']


In [173]:
# crypto_df.head(200)
# The problem with crypto historic data is taht diferent start times of the data(begining of the trading period is diffferent for each crypto)  

In [174]:
### VARIATION 2
# Define the a dictionary of lists for each asset class
# api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD']
#             ,'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC']
#             ,'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']
           
#            }

# # Store data for each asset class in a dictionary
# data = {}
# for asset_class in api_pull:
#     for ticker in api_pull[asset_class]:
#         data[asset] = pd.DataFrame(yf.download(asset, start='2010-01-10', end ='2023-01-01'))

            

In [175]:
# run streamlit
streamlit run main_strealit.py

SyntaxError: invalid syntax (3406664078.py, line 2)

In [176]:
# VARIATION 3
# Define the lists of asset classes and their respective tickers. Set it as variables, so the selection can be changed easier if neded.
api_pull = {'crypto' : ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'MATIC-USD', 'AVAX-USD', 'SOL-USD', 'TRX-USD', 'ATOM-USD', 'UNI7083-USD', 'LINK-USD'],
            'stocks' : ['AMZN', 'AAPL', 'TSLA', 'GOOGL', 'NVDA', 'MSFT', 'TSM', 'META','XOM', 'LAC'],
            'commodities' : ['GC=F', 'SI=F', 'CL=F', 'HG=F', 'LBS=F', 'ZS=F', 'GF=F', 'KE=F', 'CT=F', 'ZR=F']}




In [177]:
# Prompt user to choose assets in asset classes 
crypto = st.multiselect("Choose cryptocurrencies:", options=api_pull['crypto'])
stocks = st.multiselect("Choose stocks:", options=api_pull['stocks'])
commodities = st.multiselect("Choose commodities:", options=api_pull['commodities'])

# Calculate the number of selected tickers for each asset class
crypto_num_selected_tickers = len(api_pull.get('crypto'))
# stocks_num_selected_tickers = len(stocks)
# commodities_num_selected_tickers = len(commodities)

In [178]:
crypto_num_selected_tickers

10

In [179]:
# Store data for each asset class in a dictionary
# Use .items()to iterate over the key-value pairs of the api_pull dictionary
# pull data for the number of years requested by the user
data = {}
for asset_class, tickers in api_pull.items():
    data[asset_class] = {}
    for ticker in tickers:
        data[asset_class][ticker] = yf.Ticker(ticker).history(period=f"{num_years}y")

In [180]:
# Review pulled data:
data

{'crypto': {'BTC-USD':                                    Open          High           Low         Close       Volume  Dividends  Stock Splits
  Date                                                                                                                   
  2019-02-14 00:00:00+00:00   3631.170166   3646.256592   3607.697754   3616.880859   6271044418        0.0           0.0
  2019-02-15 00:00:00+00:00   3617.368408   3647.795166   3608.206543   3620.810791   6091952231        0.0           0.0
  2019-02-16 00:00:00+00:00   3615.270264   3652.841309   3615.270264   3629.787598   5934744052        0.0           0.0
  2019-02-17 00:00:00+00:00   3633.359375   3680.537354   3619.182129   3673.836182   7039512503        0.0           0.0
  2019-02-18 00:00:00+00:00   3671.369873   3936.665039   3669.982422   3915.714355   9908216640        0.0           0.0
  ...                                 ...           ...           ...           ...          ...        ...           ...
  2

In [147]:
# Create a data frame for each asset class with the close column
# Access the 'Close' column of the data for each asset class
# Extract the values of the data dictionary and then access the 'Close' column of each dataframe.
crypto_df = pd.DataFrame({ticker: data['crypto'][ticker]['Close'] for ticker in api_pull['crypto']})
stocks_df = pd.DataFrame({ticker: data['stocks'][ticker]['Close'] for ticker in api_pull['stocks']})
commodities_df = pd.DataFrame({ticker: data['commodities'][ticker]['Close'] for ticker in api_pull['commodities']})


In [148]:
# Rewiev df
crypto_df

,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2019-02-14 00:00:00+00:00,3616.880859,121.390205,0.001897,NaN,NaN,NaN,0.023726,NaN,NaN,0.428376
2019-02-15 00:00:00+00:00,3620.810791,122.100388,0.001908,NaN,NaN,NaN,0.024056,NaN,NaN,0.432969
2019-02-16 00:00:00+00:00,3629.787598,123.260216,0.001924,NaN,NaN,NaN,0.023954,NaN,NaN,0.440511
2019-02-17 00:00:00+00:00,3673.836182,133.598877,0.001992,NaN,NaN,NaN,0.023818,NaN,NaN,0.457499
2019-02-18 00:00:00+00:00,3915.714355,146.098373,0.002114,NaN,NaN,NaN,0.024984,NaN,NaN,0.474333
...,...,...,...,...,...,...,...,...,...,...
2023-02-10 00:00:00+00:00,21651.183594,1514.869141,0.081090,1.230595,17.798531,20.130110,0.063323,13.709134,6.360616,6.902424
2023-02-11 00:00:00+00:00,21870.875000,1539.926758,0.082372,1.251412,18.029085,20.830095,0.063805,13.758607,6.483412,7.006326
2023-02-12 00:00:00+00:00,21788.203125,1515.033691,0.082144,1.238516,17.768698,21.489979,0.063837,13.537373,6.343625,6.807924


In [149]:
# Rewiev df
stocks_df

,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2019-02-15 00:00:00-05:00,80.397499,41.234722,20.525333,55.981499,39.046783,103.676628,34.137424,162.500000,62.437393,3.130000
2019-02-19 00:00:00-05:00,81.378998,41.358112,20.375999,56.325500,38.873062,103.628700,34.469040,162.289993,62.855198,3.160000
2019-02-20 00:00:00-05:00,81.105003,41.624271,20.170668,56.029499,39.347073,103.089928,34.953003,162.559998,63.096233,3.490000
2019-02-21 00:00:00-05:00,80.972000,41.389572,19.415333,55.210499,38.657158,105.264305,34.935074,160.039993,62.525772,3.660000
2019-02-22 00:00:00-05:00,81.578003,41.851711,19.647333,55.827999,39.505898,106.765190,35.302532,161.889999,63.007843,3.770000
...,...,...,...,...,...,...,...,...,...,...
2023-02-08 00:00:00-05:00,100.050003,151.688400,201.289993,99.370003,222.050003,266.730011,94.279999,183.429993,113.050087,25.540001
2023-02-09 00:00:00-05:00,98.239998,150.639999,207.320007,95.010002,223.369995,263.619995,96.690002,177.919998,113.476807,25.799999
2023-02-10 00:00:00-05:00,97.610001,151.009995,196.889999,94.570000,212.649994,263.100006,95.370003,174.149994,118.260002,24.549999


In [150]:
# Rewiev df
commodities_df


,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2019-02-14 00:00:00-05:00,1309.800049,15.500000,54.410000,2.7760,423.299988,903.50,144.074997,481.50,70.129997,1018.500000
2019-02-15 00:00:00-05:00,1318.099976,15.723000,55.590000,2.8025,408.299988,907.50,142.600006,476.50,70.220001,999.500000
2019-02-19 00:00:00-05:00,1340.099976,15.947000,56.090000,2.8815,397.399994,900.75,143.750000,460.75,70.410004,986.500000
2019-02-20 00:00:00-05:00,1343.300049,16.157000,56.919998,2.9300,389.000000,902.50,143.875000,450.00,70.279999,1003.500000
2019-02-21 00:00:00-05:00,1323.500000,15.791000,56.959999,2.9025,398.200012,911.00,143.225006,455.25,72.190002,1021.000000
...,...,...,...,...,...,...,...,...,...,...
2023-02-08 00:00:00-05:00,1877.400024,22.367001,78.470001,4.0435,454.100006,1519.75,186.449997,896.00,85.370003,1790.000000
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500000
2023-02-10 00:00:00-05:00,1862.800049,22.034000,79.720001,4.0275,419.600006,1542.50,186.399994,909.00,85.269997,1808.500000


In [152]:
# Calculate .log() returns over time calculate .pct_change() for each asset class
crypto_log_returns = np.log(1 + crypto_df.pct_change())
stocks_log_returns = np.log(1 + stocks_df.pct_change())
commodities_log_returns = np.log(1 + commodities_df.pct_change())



In [153]:
crypto_log_returns


,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2019-02-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-15 00:00:00+00:00,0.001086,0.005833,0.005782,NaN,NaN,NaN,0.013813,NaN,NaN,0.010665
2019-02-16 00:00:00+00:00,0.002476,0.009454,0.008351,NaN,NaN,NaN,-0.004249,NaN,NaN,0.017269
2019-02-17 00:00:00+00:00,0.012062,0.080544,0.034733,NaN,NaN,NaN,-0.005694,NaN,NaN,0.037839
2019-02-18 00:00:00+00:00,0.063761,0.089438,0.059443,NaN,NaN,NaN,0.047794,NaN,NaN,0.036135
...,...,...,...,...,...,...,...,...,...,...
2023-02-10 00:00:00+00:00,-0.007723,-0.020625,-0.004798,0.001832,-0.004257,-0.015870,0.011260,0.015213,0.006862,-0.011626
2023-02-11 00:00:00+00:00,0.010096,0.016406,0.015686,0.016775,0.012870,0.034182,0.007583,0.003602,0.019122,0.014941
2023-02-12 00:00:00+00:00,-0.003787,-0.016297,-0.002772,-0.010359,-0.014548,0.031188,0.000501,-0.016210,-0.021797,-0.028726


In [154]:
stocks_log_returns


,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2019-02-15 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-19 00:00:00-05:00,0.012134,0.002988,-0.007302,0.006126,-0.004459,-0.000462,0.009667,-0.001293,0.006669,0.009539
2019-02-20 00:00:00-05:00,-0.003373,0.006415,-0.010128,-0.005269,0.012120,-0.005213,0.013943,0.001662,0.003827,0.099330
2019-02-21 00:00:00-05:00,-0.001641,-0.005654,-0.038166,-0.014725,-0.017690,0.020873,-0.000513,-0.015623,-0.009082,0.047561
2019-02-22 00:00:00-05:00,0.007456,0.011104,0.011879,0.011122,0.021718,0.014158,0.010463,0.011493,0.007680,0.029612
...,...,...,...,...,...,...,...,...,...,...
2023-02-08 00:00:00-05:00,-0.020381,-0.017810,0.022508,-0.079942,0.001442,-0.003107,-0.002860,-0.043681,-0.008740,-0.013611
2023-02-09 00:00:00-05:00,-0.018257,-0.006936,0.029517,-0.044868,0.005927,-0.011728,0.025241,-0.030499,0.003768,0.010129
2023-02-10 00:00:00-05:00,-0.006433,0.002453,-0.051618,-0.004642,-0.049182,-0.001974,-0.013746,-0.021417,0.041287,-0.049663


In [155]:
commodities_log_returns

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2019-02-14 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-15 00:00:00-05:00,0.006317,0.014285,0.021455,0.009501,-0.036079,0.004417,-0.010290,-0.010439,0.001283,-0.018831
2019-02-19 00:00:00-05:00,0.016553,0.014146,0.008954,0.027799,-0.027059,-0.007466,0.008032,-0.033612,0.002702,-0.013092
2019-02-20 00:00:00-05:00,0.002385,0.013083,0.014689,0.016691,-0.021364,0.001941,0.000869,-0.023608,-0.001848,0.017086
2019-02-21 00:00:00-05:00,-0.014850,-0.022913,0.000703,-0.009430,0.023375,0.009374,-0.004528,0.011599,0.026814,0.017289
...,...,...,...,...,...,...,...,...,...,...
2023-02-08 00:00:00-05:00,0.003041,0.011240,0.017094,-0.011068,-0.033351,0.002965,-0.004014,0.011506,-0.003041,0.024029
2023-02-09 00:00:00-05:00,-0.005984,-0.012145,-0.005239,0.015217,-0.046195,-0.000329,0.002009,-0.019156,0.001522,0.011940
2023-02-10 00:00:00-05:00,-0.001823,-0.002855,0.021043,-0.019182,-0.032821,0.015188,-0.002277,0.033560,-0.002694,-0.001657


In [156]:
# Generate random weights for each asset class
crypto_random_wts

In [127]:
# Replace all the NaN values in a data frame using the fillna() method. Create a copy of the data frames.
cryptodf_df = crypto_df.fillna(0).copy()
dstocksf_df = df_stocksf_df.fillna(0).copy()
commoditiesf_df = commoditiesf_df.fillna(0).copy()

In [108]:
# Review first and last 10 rows for crypto assets class
display(cryptof_df.head(10))
cryptof_df.tail(10)



,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2019-02-14 00:00:00+00:00,3616.880859,121.390205,0.001897,0.0,0.0,0.0,0.023726,0.0,0.0,0.428376
2019-02-15 00:00:00+00:00,3620.810791,122.100388,0.001908,0.0,0.0,0.0,0.024056,0.0,0.0,0.432969
2019-02-16 00:00:00+00:00,3629.787598,123.260216,0.001924,0.0,0.0,0.0,0.023954,0.0,0.0,0.440511
2019-02-17 00:00:00+00:00,3673.836182,133.598877,0.001992,0.0,0.0,0.0,0.023818,0.0,0.0,0.457499
2019-02-18 00:00:00+00:00,3915.714355,146.098373,0.002114,0.0,0.0,0.0,0.024984,0.0,0.0,0.474333
2019-02-19 00:00:00+00:00,3947.094482,145.346191,0.002067,0.0,0.0,0.0,0.024993,0.0,0.0,0.458156
2019-02-20 00:00:00+00:00,3999.820557,149.554337,0.002079,0.0,0.0,0.0,0.025228,0.0,0.0,0.466110
2019-02-21 00:00:00+00:00,3954.118164,146.130844,0.002028,0.0,0.0,0.0,0.024804,0.0,0.0,0.445463
2019-02-22 00:00:00+00:00,4005.526611,149.092453,0.002056,0.0,0.0,0.0,0.025158,0.0,0.0,0.452029


,BTC-USD,ETH-USD,DOGE-USD,MATIC-USD,AVAX-USD,SOL-USD,TRX-USD,ATOM-USD,UNI7083-USD,LINK-USD
Date,,,,,,,,,,
2023-02-05 00:00:00+00:00,22955.666016,1631.645874,0.092095,1.204501,20.108017,23.493439,0.064085,14.754556,6.899310,6.984616
2023-02-06 00:00:00+00:00,22760.109375,1616.247070,0.089549,1.190415,19.585035,22.706388,0.063194,14.142922,6.652495,6.832168
2023-02-07 00:00:00+00:00,23264.291016,1672.003540,0.092576,1.268975,20.845219,23.861357,0.065774,14.957318,6.984529,7.198653
2023-02-08 00:00:00+00:00,22939.398438,1650.716797,0.090156,1.303970,20.081474,23.192671,0.066874,15.146101,6.799746,7.121416
2023-02-09 00:00:00+00:00,21819.039062,1546.438232,0.081480,1.228342,17.874453,20.452127,0.062614,13.502150,6.317118,6.983141
2023-02-10 00:00:00+00:00,21651.183594,1514.869141,0.081090,1.230595,17.798531,20.130110,0.063323,13.709134,6.360616,6.902424
2023-02-11 00:00:00+00:00,21870.875000,1539.926758,0.082372,1.251412,18.029085,20.830095,0.063805,13.758607,6.483412,7.006326
2023-02-12 00:00:00+00:00,21788.203125,1515.033691,0.082144,1.238516,17.768698,21.489979,0.063837,13.537373,6.343625,6.807924
2023-02-13 00:00:00+00:00,21808.101562,1507.165894,0.082156,1.186845,17.620972,20.790100,0.065600,13.140862,6.254118,6.671660


In [103]:
# Review first and last 10 rows for stocks assets class
display(stocksf_df.head(10))
stocksf_df.tail(10)

,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2019-02-15 00:00:00-05:00,80.397499,41.234715,20.525333,55.981499,39.046787,103.676613,34.137428,162.500000,62.437386,3.13
2019-02-19 00:00:00-05:00,81.378998,41.358120,20.375999,56.325500,38.873066,103.628716,34.469036,162.289993,62.855209,3.16
2019-02-20 00:00:00-05:00,81.105003,41.624260,20.170668,56.029499,39.347069,103.089943,34.953003,162.559998,63.096226,3.49
2019-02-21 00:00:00-05:00,80.972000,41.389561,19.415333,55.210499,38.657162,105.264290,34.935070,160.039993,62.525772,3.66
2019-02-22 00:00:00-05:00,81.578003,41.851711,19.647333,55.827999,39.505894,106.765190,35.302532,161.889999,63.007832,3.77
2019-02-25 00:00:00-05:00,81.650002,42.156578,19.917999,55.866501,39.381809,107.361687,35.472809,164.619995,63.072117,3.76
2019-02-26 00:00:00-05:00,81.820000,42.180779,19.857332,56.100498,38.987221,108.102501,35.490734,164.130005,63.200691,3.61
2019-02-27 00:00:00-05:00,82.054497,42.311436,20.982668,56.144501,38.567822,107.919716,35.356300,162.809998,63.851482,3.64
2019-02-28 00:00:00-05:00,81.991501,41.895267,21.325333,56.327499,38.321880,107.785019,34.997814,161.449997,63.497974,3.70


,AMZN,AAPL,TSLA,GOOGL,NVDA,MSFT,TSM,META,XOM,LAC
Date,,,,,,,,,,
2023-02-01 00:00:00-05:00,105.150002,145.208282,181.410004,100.430000,209.429993,252.750000,94.739998,153.119995,113.863831,24.700001
2023-02-02 00:00:00-05:00,112.910004,150.590088,188.270004,107.739998,217.089996,264.600006,96.650002,188.770004,110.301247,24.959999
2023-02-03 00:00:00-05:00,103.389999,154.264465,189.979996,104.779999,211.000000,258.350006,94.660004,186.529999,111.065361,24.320000
2023-02-06 00:00:00-05:00,102.180000,151.498688,194.759995,102.900002,210.889999,256.769989,91.830002,186.059998,110.876816,23.459999
2023-02-07 00:00:00-05:00,102.110001,154.414230,196.809998,107.639999,221.729996,267.559998,94.550003,191.619995,114.042458,25.889999
2023-02-08 00:00:00-05:00,100.050003,151.688400,201.289993,99.370003,222.050003,266.730011,94.279999,183.429993,113.050087,25.540001
2023-02-09 00:00:00-05:00,98.239998,150.639999,207.320007,95.010002,223.369995,263.619995,96.690002,177.919998,113.476807,25.799999
2023-02-10 00:00:00-05:00,97.610001,151.009995,196.889999,94.570000,212.649994,263.100006,95.370003,174.149994,118.260002,24.549999
2023-02-13 00:00:00-05:00,99.540001,153.850006,194.639999,94.610001,217.880005,271.320007,96.110001,179.429993,117.800003,24.129999


In [104]:
# Review first and last 10 rows for commodities assets class
display(commoditiesf_df.head(10))
commoditiesf_df.tail(10)

,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2019-02-14 00:00:00-05:00,1309.800049,15.500,54.410000,2.7760,423.299988,903.50,144.074997,481.50,70.129997,1018.5
2019-02-15 00:00:00-05:00,1318.099976,15.723,55.590000,2.8025,408.299988,907.50,142.600006,476.50,70.220001,999.5
2019-02-19 00:00:00-05:00,1340.099976,15.947,56.090000,2.8815,397.399994,900.75,143.750000,460.75,70.410004,986.5
2019-02-20 00:00:00-05:00,1343.300049,16.157,56.919998,2.9300,389.000000,902.50,143.875000,450.00,70.279999,1003.5
2019-02-21 00:00:00-05:00,1323.500000,15.791,56.959999,2.9025,398.200012,911.00,143.225006,455.25,72.190002,1021.0
2019-02-22 00:00:00-05:00,1329.199951,15.908,57.259998,2.9610,396.899994,910.25,142.899994,458.50,71.839996,1019.0
2019-02-25 00:00:00-05:00,1325.900024,15.825,55.480000,2.9500,395.899994,911.75,142.500000,442.25,71.930000,1010.0
2019-02-26 00:00:00-05:00,1325.099976,15.827,55.500000,2.9440,388.700012,903.75,143.449997,435.75,70.849998,1036.5
2019-02-27 00:00:00-05:00,1317.699951,15.672,56.939999,2.9590,388.700012,903.50,143.875000,438.75,71.190002,1028.5


,GC=F,SI=F,CL=F,HG=F,LBS=F,ZS=F,GF=F,KE=F,CT=F,ZR=F
Date,,,,,,,,,,
2023-02-01 00:00:00-05:00,1927.800049,23.521000,76.410004,4.1110,509.600006,1520.25,183.250000,884.25,85.610001,1784.500000
2023-02-02 00:00:00-05:00,1916.300049,23.534000,75.879997,4.0940,512.200012,1534.25,185.925003,880.75,86.389999,1778.000000
2023-02-03 00:00:00-05:00,1862.900024,22.334000,73.389999,4.0630,499.799988,1532.00,186.100006,873.00,85.430000,1774.000000
2023-02-06 00:00:00-05:00,1866.199951,22.177000,74.110001,4.0435,486.700012,1521.25,187.699997,876.00,83.269997,1754.500000
2023-02-07 00:00:00-05:00,1871.699951,22.117001,77.139999,4.0885,469.500000,1515.25,187.199997,885.75,85.629997,1747.500000
2023-02-08 00:00:00-05:00,1877.400024,22.367001,78.470001,4.0435,454.100006,1519.75,186.449997,896.00,85.370003,1790.000000
2023-02-09 00:00:00-05:00,1866.199951,22.097000,78.059998,4.1055,433.600006,1519.25,186.824997,879.00,85.500000,1811.500000
2023-02-10 00:00:00-05:00,1862.800049,22.034000,79.720001,4.0275,419.600006,1542.50,186.399994,909.00,85.269997,1808.500000
2023-02-13 00:00:00-05:00,1851.900024,21.818001,80.139999,4.0870,414.100006,1542.75,187.199997,912.25,85.639999,1791.000000


## Step 3 : 
### Prepare data ready for the Monte Carlo simulation
### ??? Should we prompt user to enter the number of simulations for the model ???
### !!! Keep in mind that for 'stocks' and 'comodities' trading_days = 252, but for 'crypto' trading_days = 365.!!!

In [83]:
## Prepare the data for Monte Carlo simulation
returns = portfolio_data.pct_change() #Calculate returns for each asset
mean_return = returns.mean()
cov_matrix = returns.cov()

NameError: name 'portfolio_data' is not defined

In [ ]:
# Set the number of simulations and trading days
num_simulations = 1000
stocks_n_comodities_num_trading_days = 252
crypto_num_trading_days = 365

## Step 4 : 
### Plot an overlay of the closing prices for each asset class (10 assetset each)
### ??? Plot the returns ???


In [ ]:

# Limit the number of securities for each alloocation( depends on the data fed)

# Get combination of alocated assets and weighting 
# Use KMeans algorithm for clustering 
#from sklearn.cluster import KMeans
#import numpy as np






In [ ]:
#Data from 3 assets with their weight amounts
#Limit each equity t0 allocate 10 items 
assets = {'equity':range(0,equity_alloc,10),'commodity':range(0,commodity_alloc,10),'crypto':range(0,crypto_alloc,10)}




In [ ]:
#Allowing only 3 cluster of assets 
kmeans = KMeans(n_clusters=3)



In [ ]:
#Fit the model to the asset data 
kmeans.fit(assets)



In [ ]:
# Plot vizualization fort the clusters 

#### For streamlit install pip install streamlit==0.62.0
### there is a problem problem was related to Python version.that pyarrow is a requirement in streamlit version 0.63.0. So you can use pip install streamlit==0.62.0 and it will work just fine